In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import sklearn
import seaborn as sns
from sklearn import metrics
from sklearn.cross_validation import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm
%matplotlib inline
style.use("ggplot")

accuracy = metrics.accuracy_score

/Users/kerrylam/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv("titanic_training.csv", header = 0) 

In [3]:
del train['Name'], train['Ticket'], train['Cabin'], train['PassengerId']

In [4]:
train = pd.get_dummies(train, columns = ['Sex', 'Embarked', 'Pclass'], drop_first = True)
train.head()

,Survived,Age,SibSp,Parch,Fare,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,0,22.0,1,0,7.2500,1,0,0,1,0,1
1,1,38.0,1,0,71.2833,0,1,0,0,0,0
2,1,26.0,0,0,7.9250,0,0,0,1,0,1
3,1,35.0,1,0,53.1000,0,0,0,1,0,0
4,0,35.0,0,0,8.0500,1,0,0,1,0,1


In [5]:
del train["Embarked_C"]
train.head()

,Survived,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,0,22.0,1,0,7.2500,1,0,1,0,1
1,1,38.0,1,0,71.2833,0,0,0,0,0
2,1,26.0,0,0,7.9250,0,0,1,0,1
3,1,35.0,1,0,53.1000,0,0,1,0,0
4,0,35.0,0,0,8.0500,1,0,1,0,1


In [6]:
train = train.fillna(train.mean(), inplace = True)  # Impute Missing Data
train = train.astype(float) # Convert DF Type to Float

"""Split Data"""
train = train[0:len(train)]
test = train[len(train):]

"""Normalize Data for Faster Computation"""
train = train/train.max().astype(np.float64)
test = test/test.max().astype(np.float64)

In [7]:
"""Moving Survived Column Into The Front Will Make It Easier to Extract Later"""
cols = train.columns.tolist()
cols.insert(0, cols.pop(cols.index('Survived')))
train = train.reindex(columns= cols)
train.head()

,Survived,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,0.0,0.2750,0.125,0.0,0.014151,1.0,0.0,1.0,0.0,1.0
1,1.0,0.4750,0.125,0.0,0.139136,0.0,0.0,0.0,0.0,0.0
2,1.0,0.3250,0.000,0.0,0.015469,0.0,0.0,1.0,0.0,1.0
3,1.0,0.4375,0.125,0.0,0.103644,0.0,0.0,1.0,0.0,0.0
4,0.0,0.4375,0.000,0.0,0.015713,1.0,0.0,1.0,0.0,1.0


In [8]:
print("Index Location of Target:", train.columns.get_loc("Survived"))

Index Location of Target: 0


In [9]:
"""Split The Survived Column From Features"""
X = pd.DataFrame(train.ix[:,1:])
target = train["Survived"]
X.head()

,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,0.2750,0.125,0.0,0.014151,1.0,0.0,1.0,0.0,1.0
1,0.4750,0.125,0.0,0.139136,0.0,0.0,0.0,0.0,0.0
2,0.3250,0.000,0.0,0.015469,0.0,0.0,1.0,0.0,1.0
3,0.4375,0.125,0.0,0.103644,0.0,0.0,1.0,0.0,0.0
4,0.4375,0.000,0.0,0.015713,1.0,0.0,1.0,0.0,1.0


In [10]:
"""Create Our Own Training And Testing Set To Test Our Model's Accuracy"""
train_, test_, targ_train, targ_test = train_test_split(X, target, test_size = 0.32, random_state = None)

In [11]:
print ("Train:       ", train_.shape)
print ("Train Class: ", targ_train.shape)
print ("Test:        ", test_.shape)
print ("Test Class:  ", targ_test.shape)

Train:        (605, 9)
Train Class:  (605,)
Test:         (286, 9)
Test Class:   (286,)


In [12]:
"""Reset Index: This Will Make Storing The Data Easier Later"""
train_ = train.reset_index()
test_ = test_.reset_index()
targ_test = targ_test.reset_index()

In [13]:
"""Delete The Old Indexes"""
del train_["index"], test_["index"], targ_test["index"]

### Part 1: 

Partition Training Data into 5 Folds:

- Create train_meta with same rows and IDs as training data
- Create Empty Columns: M1 and M2 (NaNs)

- Create test_meta variable with same rows and IDs as test dataset 
- Create Empty Columns: M1 and M2 (NaNs)

In [14]:
"""Create Our Train and Test Meta Sets"""
train_meta = pd.DataFrame.copy(train_)
train_meta["M1"] = np.nan
train_meta["M2"] = np.nan
train_meta["M3"] = np.nan
print(train.shape)
train_meta.head()

(891, 10)


,Survived,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3,M1,M2,M3
0,0.0,0.2750,0.125,0.0,0.014151,1.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN
1,1.0,0.4750,0.125,0.0,0.139136,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
2,1.0,0.3250,0.000,0.0,0.015469,0.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN
3,1.0,0.4375,0.125,0.0,0.103644,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN
4,0.0,0.4375,0.000,0.0,0.015713,1.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN


In [15]:
test_meta = pd.DataFrame.copy(test_)
print(test_.shape)
test_meta.head()

(286, 9)


,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,0.2000,0.00,0.0,0.050749,1.0,0.0,1.0,1.0,0.0
1,0.0000,0.00,0.0,0.016510,1.0,1.0,0.0,0.0,1.0
2,0.3125,0.00,0.0,0.015412,1.0,0.0,1.0,0.0,1.0
3,0.2625,0.25,0.0,0.143462,1.0,0.0,1.0,1.0,0.0
4,0.0000,0.00,0.0,0.069291,1.0,0.0,1.0,0.0,0.0


### Part 2

For each train fold: combine the other 4 into a training set and predict on each fold i 
- train fold 1 = 2,3,4,5 (Predict on fold 1)
- train fold 2 = 1,3,4,5 (Predict on Fold 2)
- train fold 3 = 1,2,4,5 (Predict on Fold 3)
- train fold 4 = 1,2,3,5 (Predict on Fold 4)
- train fold 5 = 1,2,3,4 (Predict on Fold 5)

In [16]:
"""This Data Will Be Stored In The Code At The Bottom"""
from sklearn.model_selection import KFold
kf = KFold(n_splits = 10, shuffle = False)
for train_index, test_index in kf.split(train_):
    print("Train:", train_index, "Test:", test_index, "\n")

Train: [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251
 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269
 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287
 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305
 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323
 324 325 326 327 328 329 330 331 332 333 334

In [17]:
print("Index Location of Target:", train.columns.get_loc("Survived"))

Index Location of Target: 0


### We Will Need To Copy and Paste Output: Change Target Index In Folds If Necessary
    - Usually Will Be The 0th Index If We Preset The Index Above

In [18]:
j = 1
# The Variable "j" Sets It So That Our First Training Set Will Be Called: train_fold1, test_fold1, target_fold1, etc..

for train_index, test_index in kf.split(train):
# Set Up To {1-5} Train Folds With All The Data
    print ('train_fold{} = train'.format(j))  
    
# Seperate Up To {1-5} Test Folds Without The Class
    print ('test_fold{} = train.ix[{}:{}, 1:]'.format(j, min(test_index), max(test_index) + 1))
    
# Seperate Up To {1-5} Target Folds Using Only The Class
    print ('target_fold{} = train_fold{}.ix[:,0]'.format(j,j))
    
# Seperate All Features Out Of The Training Set {1-5} 
    print ('train_fold{} = train_fold{}.ix[{}:,1:]'.format(j,j, 0))
    
# Drop 1/5th Of The Excluded Training Set Which Is Used As The Test Data For Cross Validation 
    print ('train_fold{} = train_fold{}.drop(train_fold{}.index[{}:{}])'.format(j, j, j, min(test_index), max(test_index) + 1))
    
# Drop 1/5th Of The Excluded Target Set Which Is Used As The Test Data For Cross Validation 
    print ('target_fold{} = target_fold{}.drop(target_fold{}.index[{}:{}])'.format(j, j, j, min(test_index), max(test_index) + 1), "\n")

# Iterate By 1. This Will Stop At The Number Of Training Folds We Have (e.g. 5 Folds)
    j += 1

train_fold1 = train
test_fold1 = train.ix[0:90, 1:]
target_fold1 = train_fold1.ix[:,0]
train_fold1 = train_fold1.ix[0:,1:]
train_fold1 = train_fold1.drop(train_fold1.index[0:90])
target_fold1 = target_fold1.drop(target_fold1.index[0:90]) 

train_fold2 = train
test_fold2 = train.ix[90:179, 1:]
target_fold2 = train_fold2.ix[:,0]
train_fold2 = train_fold2.ix[0:,1:]
train_fold2 = train_fold2.drop(train_fold2.index[90:179])
target_fold2 = target_fold2.drop(target_fold2.index[90:179]) 

train_fold3 = train
test_fold3 = train.ix[179:268, 1:]
target_fold3 = train_fold3.ix[:,0]
train_fold3 = train_fold3.ix[0:,1:]
train_fold3 = train_fold3.drop(train_fold3.index[179:268])
target_fold3 = target_fold3.drop(target_fold3.index[179:268]) 

train_fold4 = train
test_fold4 = train.ix[268:357, 1:]
target_fold4 = train_fold4.ix[:,0]
train_fold4 = train_fold4.ix[0:,1:]
train_fold4 = train_fold4.drop(train_fold4.index[268:357])
target_fold4 = target_fold4.drop(target_fold4.index[268:357]) 

train_fold5 = tr

In [19]:
train_fold1 = train
test_fold1 = train.ix[0:90, 1:]
target_fold1 = train_fold1.ix[:,0]
train_fold1 = train_fold1.ix[0:,1:]
train_fold1 = train_fold1.drop(train_fold1.index[0:90])
target_fold1 = target_fold1.drop(target_fold1.index[0:90]) 

train_fold2 = train
test_fold2 = train.ix[90:179, 1:]
target_fold2 = train_fold2.ix[:,0]
train_fold2 = train_fold2.ix[0:,1:]
train_fold2 = train_fold2.drop(train_fold2.index[90:179])
target_fold2 = target_fold2.drop(target_fold2.index[90:179]) 

train_fold3 = train
test_fold3 = train.ix[179:268, 1:]
target_fold3 = train_fold3.ix[:,0]
train_fold3 = train_fold3.ix[0:,1:]
train_fold3 = train_fold3.drop(train_fold3.index[179:268])
target_fold3 = target_fold3.drop(target_fold3.index[179:268]) 

train_fold4 = train
test_fold4 = train.ix[268:357, 1:]
target_fold4 = train_fold4.ix[:,0]
train_fold4 = train_fold4.ix[0:,1:]
train_fold4 = train_fold4.drop(train_fold4.index[268:357])
target_fold4 = target_fold4.drop(target_fold4.index[268:357]) 

train_fold5 = train
test_fold5 = train.ix[357:446, 1:]
target_fold5 = train_fold5.ix[:,0]
train_fold5 = train_fold5.ix[0:,1:]
train_fold5 = train_fold5.drop(train_fold5.index[357:446])
target_fold5 = target_fold5.drop(target_fold5.index[357:446]) 

train_fold6 = train
test_fold6 = train.ix[446:535, 1:]
target_fold6 = train_fold6.ix[:,0]
train_fold6 = train_fold6.ix[0:,1:]
train_fold6 = train_fold6.drop(train_fold6.index[446:535])
target_fold6 = target_fold6.drop(target_fold6.index[446:535]) 

train_fold7 = train
test_fold7 = train.ix[535:624, 1:]
target_fold7 = train_fold7.ix[:,0]
train_fold7 = train_fold7.ix[0:,1:]
train_fold7 = train_fold7.drop(train_fold7.index[535:624])
target_fold7 = target_fold7.drop(target_fold7.index[535:624]) 

train_fold8 = train
test_fold8 = train.ix[624:713, 1:]
target_fold8 = train_fold8.ix[:,0]
train_fold8 = train_fold8.ix[0:,1:]
train_fold8 = train_fold8.drop(train_fold8.index[624:713])
target_fold8 = target_fold8.drop(target_fold8.index[624:713]) 

train_fold9 = train
test_fold9 = train.ix[713:802, 1:]
target_fold9 = train_fold9.ix[:,0]
train_fold9 = train_fold9.ix[0:,1:]
train_fold9 = train_fold9.drop(train_fold9.index[713:802])
target_fold9 = target_fold9.drop(target_fold9.index[713:802]) 

train_fold10 = train
test_fold10 = train.ix[802:891, 1:]
target_fold10 = train_fold10.ix[:,0]
train_fold10 = train_fold10.ix[0:,1:]
train_fold10 = train_fold10.drop(train_fold10.index[802:891])
target_fold10 = target_fold10.drop(target_fold10.index[802:891])

In [20]:
print(train_fold1.shape)
print(target_fold1.shape)
print(test_fold1.shape) 

print(train_fold2.shape)
print(target_fold2.shape)
print(test_fold2.shape) 

print(train_fold3.shape)
print(target_fold3.shape)
print(test_fold3.shape) 

print(train_fold4.shape)
print(target_fold4.shape)
print(test_fold4.shape) 

print(train_fold5.shape)
print(target_fold5.shape)
print(test_fold5.shape) 

(801, 9)
(801,)
(91, 9)
(802, 9)
(802,)
(90, 9)
(802, 9)
(802,)
(90, 9)
(802, 9)
(802,)
(90, 9)
(802, 9)
(802,)
(90, 9)


### Part 3
- Use Model M1 to train on each train fold i and predict the excluded fold and store the values back into M1 for its fold ID
- Do the same with M2

In [21]:
"""Instantiate Models"""
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier

clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1, n_estimators = 100)
clf3 = GaussianNB()

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='soft')

est = KNeighborsClassifier(n_neighbors = 15)

m1 = BaggingClassifier(base_estimator=est, 
                        n_estimators = 10, 
                        max_samples = 0.35,
                        max_features = 8,
                        random_state = 1, 
                        oob_score = True)
m2 = GradientBoostingClassifier(n_estimators = 100)
m3 = eclf

In [22]:
"""Train Data On First Model"""
train_folds = [train_fold1, train_fold2, train_fold3, train_fold4, train_fold5, train_fold6, train_fold7, train_fold8, train_fold9, train_fold10]
target_folds = [target_fold1, target_fold2, target_fold3, target_fold4, target_fold5, target_fold6, target_fold7, target_fold8, target_fold9, target_fold10]
test_folds = [test_fold1, test_fold2, test_fold3, test_fold4, test_fold5, test_fold6, test_fold7, test_fold8, test_fold9, test_fold10]

"""Create An Empty List That Will Join The Outputs From Each Fold And Store Them Into M1 Column"""
y_pred1 = []
for i, j in zip(train_folds, target_folds):
    m1.fit(i, j)
for k in test_folds:
    target_pred = m1.predict(k)
    y_pred1.append(target_pred) 
    
y_pred1 = np.concatenate(y_pred1, axis=0)
y_pred1 = pd.DataFrame(y_pred1)
train_meta["M1"] = y_pred1

In [23]:
"""Train Data On Second Model"""
y_pred2 = []
for i, j in zip(train_folds, target_folds):
    m2.fit(i, j)
for k in test_folds:
    target_pred2 = m2.predict(k)
    y_pred2.append(target_pred2) 
    
y_pred2 = np.concatenate(y_pred2, axis=0)
y_pred2 = pd.DataFrame(y_pred2)
train_meta["M2"] = y_pred2

In [24]:
"""Train Data On Third Model"""
y_pred3 = []
for i, j in zip(train_folds, target_folds):
    m3.fit(i, j)
for k in test_folds:
    target_pred3 = m3.predict(k)
    y_pred3.append(target_pred3) 
    
y_pred3 = np.concatenate(y_pred3, axis=0)
y_pred3 = pd.DataFrame(y_pred3)
train_meta["M3"] = y_pred3

### Part 4

Fit m1 Model to train_meta
- Put new predictions into train_meta m1 column
Fit m2 Model to train_meta
- Put new predictions into train_meta m2 column

In [25]:
train_meta

,Survived,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3,M1,M2,M3
0,0.0,0.2750,0.125,0.000000,0.014151,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.4750,0.125,0.000000,0.139136,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2,1.0,0.3250,0.000,0.000000,0.015469,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
3,1.0,0.4375,0.125,0.000000,0.103644,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
4,0.0,0.4375,0.000,0.000000,0.015713,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
5,0.0,0.0000,0.000,0.000000,0.016510,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
6,0.0,0.6750,0.000,0.000000,0.101229,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0250,0.375,0.166667,0.041136,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
8,1.0,0.3375,0.000,0.333333,0.021731,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
9,1.0,0.1750,0.125,0.000000,0.058694,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0


### Part 5
- Predict m1 on test_meta
- Predict m2 on test_meta

Part 4
Use another model (S) and use train_meta to make final predictions on test_meta 

In [26]:
train_meta.head()

,Survived,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3,M1,M2,M3
0,0.0,0.2750,0.125,0.0,0.014151,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.4750,0.125,0.0,0.139136,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2,1.0,0.3250,0.000,0.0,0.015469,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
3,1.0,0.4375,0.125,0.0,0.103644,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
4,0.0,0.4375,0.000,0.0,0.015713,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


### Store Our New Targets (M1, M2, Survived) From Our Training Set Into New Target Variables
    - Delete Them Off Training Set
    - These Will Be Used To Fit Out Models To Train On Our Test Data
    - Before Deleting Them Off, Make A Copy For When We Train On Our S Model

In [27]:
train_meta_Class = train_meta["Survived"]
train_meta_M1 = train_meta["M1"]
train_meta_M2 = train_meta["M2"]
train_meta_M3 = train_meta["M3"]

del train_meta["Survived"]

"""Create A New Training Set For Predictions Of Test Data With M1, M2, M3 Columns"""
training = pd.DataFrame.copy(train_meta)
del train_meta["M1"], train_meta["M2"], train_meta["M3"]

In [28]:
training.head()

,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3,M1,M2,M3
0,0.2750,0.125,0.0,0.014151,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.4750,0.125,0.0,0.139136,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2,0.3250,0.000,0.0,0.015469,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
3,0.4375,0.125,0.0,0.103644,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
4,0.4375,0.000,0.0,0.015713,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [29]:
train_meta.head()

,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,0.2750,0.125,0.0,0.014151,1.0,0.0,1.0,0.0,1.0
1,0.4750,0.125,0.0,0.139136,0.0,0.0,0.0,0.0,0.0
2,0.3250,0.000,0.0,0.015469,0.0,0.0,1.0,0.0,1.0
3,0.4375,0.125,0.0,0.103644,0.0,0.0,1.0,0.0,0.0
4,0.4375,0.000,0.0,0.015713,1.0,0.0,1.0,0.0,1.0


In [30]:
"""Train Data On Third Model"""
m1.fit(train_meta, train_meta_M1)
target_pred_M1 = m1.predict(test_meta)

m2.fit(train_meta, train_meta_M2)
target_pred_M2 = m2.predict(test_meta)

m3.fit(train_meta, train_meta_M3)
target_pred_M3 = m3.predict(test_meta)

test_meta["M1"] = target_pred_M1
test_meta["M2"] = target_pred_M2
test_meta["M3"] = target_pred_M3

In [31]:
test_meta.head()

,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3,M1,M2,M3
0,0.2000,0.00,0.0,0.050749,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,0.0000,0.00,0.0,0.016510,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.3125,0.00,0.0,0.015412,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.2625,0.25,0.0,0.143462,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,0.0000,0.00,0.0,0.069291,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### Part 6 Predict On Actual Testing Set

In [32]:
training.head()

,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3,M1,M2,M3
0,0.2750,0.125,0.0,0.014151,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.4750,0.125,0.0,0.139136,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2,0.3250,0.000,0.0,0.015469,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
3,0.4375,0.125,0.0,0.103644,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
4,0.4375,0.000,0.0,0.015713,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [33]:
train_meta_Class.head()

0    0.0
1    1.0
2    1.0
3    1.0
4    0.0
Name: Survived, dtype: float64

In [34]:
test_meta.head()

,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3,M1,M2,M3
0,0.2000,0.00,0.0,0.050749,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,0.0000,0.00,0.0,0.016510,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.3125,0.00,0.0,0.015412,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.2625,0.25,0.0,0.143462,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,0.0000,0.00,0.0,0.069291,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### Using Stacking Model

In [35]:
s = GradientBoostingClassifier(n_estimators = 100)
s.fit(training, train_meta_Class)
target_pred_Class = s.predict(test_meta)

test_meta["Survived"] = target_pred_Class

In [36]:
print(round(accuracy(targ_test, target_pred_Class)*100, 2), "% Accuracy")

86.36 % Accuracy


In [37]:
test_meta

,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3,M1,M2,M3,Survived
0,0.200000,0.000,0.000000,0.050749,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000,0.000000,0.016510,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.312500,0.000,0.000000,0.015412,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.262500,0.250,0.000000,0.143462,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000,0.000000,0.069291,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.000,0.000000,0.054107,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.700000,0.000,0.000000,0.069291,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.000000,0.125,0.166667,0.029758,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,0.637500,0.000,0.000000,0.024447,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
9,0.009375,0.250,0.166667,0.037590,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


### Testing Without Other Features

In [38]:
# trainClasses = training.ix[:,9:]
# testClasses = test_meta.ix[:,9:]

In [39]:
# s = LogisticRegression()
# s.fit(trainClasses, train_meta_Class)
# target_pred_Class = s.predict(testClasses)

# test_meta["Survived"] = target_pred_Class

In [40]:
# print(round(accuracy(targ_test, target_pred_Class)*100, 2), "% Accuracy")

In [41]:
# test_meta["Actual"] = targ_test

In [42]:
# test_meta